---
title: Assignment 04
subtitle: "Regression Analysis in Pyspark"
author:
  - name: Tracy Anyasi
    affiliations:
      - id: bu
        name: Boston University
        city: Boston
        state: MA
number-sections: true
date: '2025-10-08'
date-modified: today
date-format: long
format:
  html:
    theme: cerulean
    toc: true
    toc-depth: 2

execute:
  echo: false
  eval: false
  freeze: auto
---

In [1]:
from pyspark.sql import SparkSession
import pandas as pd
import plotly.express as px
import plotly.io as pio
import numpy as np

np.random.seed(42)

pio.renderers.default = "notebook+notebook_connected+vscode"

# Initialize Spark Session
spark = SparkSession.builder.appName("LightcastData").getOrCreate()

# Load Data
df = spark.read.option("header", "true").option("inferSchema", "true").option("multiLine","true").option("escape", "\"").csv("./data/lightcast_job_postings.csv")

# Show Schema and Sample Data
print("---This is Diagnostic check, No need to print it in the final doc---")

# df.printSchema() # comment this line when rendering the submission
df.show(5)

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/05 17:25:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


---This is Diagnostic check, No need to print it in the final doc---


25/10/05 17:25:47 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------------+-----------------+----------------------+----------+--------+---------+--------+--------------------+--------------------+--------------------+-----------+-------------------+--------------------+--------------------+---------------+----------------+--------+--------------------+-----------+-------------------+----------------+---------------------+-------------+-------------------+-------------+------------------+---------------+--------------------+--------------------+--------------------+-------------+------+-----------+----------------+-------------------+---------+-----------+--------------------+--------------------+-------------+------+--------------+-----+--------------------+-----+----------+---------------+--------------------+---------------+--------------------+------------+--------------------+------------+--------------------+------+--------------------+------+--------------------+------+--------------------+------+--------------------+------+------

## Feature Engineering

Remove incomplete data, keep relevant variables, and iron out complicated string values
Encoder turns categorical columns (remote, hybrid, onsite) to numeric ones (1 or 0) based on input

In [2]:
#| eval: true
#| echo: falase
#| fig-align: center

from pyspark.sql.functions import col, pow, when
from pyspark.ml.feature import StringIndexer, VectorAssembler, OneHotEncoder
from pyspark.ml import Pipeline

#remove rows with NAs
df_cleaned = df.dropna(subset=[
    "SALARY", "MIN_YEARS_EXPERIENCE", "STATE_NAME", "EMPLOYMENT_TYPE_NAME",
    "REMOTE_TYPE_NAME", "MIN_EDULEVELS_NAME", "DURATION", 
    "IS_INTERNSHIP", "COMPANY_IS_STAFFING"
])

eda_cols = [
    "SALARY", "MIN_YEARS_EXPERIENCE", "DURATION", "COMPANY_IS_STAFFING",
    "IS_INTERNSHIP", "STATE_NAME", "REMOTE_TYPE_NAME",
    "EMPLOYMENT_TYPE_NAME", "MIN_EDULEVELS_NAME"
]
df_cleaned = df_cleaned.select(eda_cols)

#clean up REMOTE_TYPE_NAME and reduce the different inputs
df_cleaned = df_cleaned.withColumn(
    "REMOTE_TYPE_NAME",
    when(col("REMOTE_TYPE_NAME") == "Remote", "Remote")
    .when(col("REMOTE_TYPE_NAME") == "[None]", "Undefined")
    .when(col("REMOTE_TYPE_NAME") == "Not Remote", "On Premise")
    .when(col("REMOTE_TYPE_NAME") == "Hybrid Remote", "Hybrid")
    .when(col("REMOTE_TYPE_NAME").isNull(), "On Premise")
    .otherwise(col("REMOTE_TYPE_NAME"))
)

#clean EMPLOYMENT_TYPE_NAME
df_cleaned = df_cleaned.withColumn(
    "EMPLOYMENT_TYPE_NAME",
    when(col("EMPLOYMENT_TYPE_NAME") == "Part-time / full-time", "Flexible")
    .when(col("EMPLOYMENT_TYPE_NAME") == "Part-time (â‰¤ 32 hours)", "Parttime")
    .when(col("EMPLOYMENT_TYPE_NAME") == "Full-time (> 32 hours)", "Fulltime")
    .when(col("EMPLOYMENT_TYPE_NAME").isNull(), "Fulltime")
    .otherwise(col("EMPLOYMENT_TYPE_NAME"))
)

# Categorical and numeric columns
categorical_cols = ["EMPLOYMENT_TYPE_NAME", "REMOTE_TYPE_NAME"]
continuous_cols = ["MIN_YEARS_EXPERIENCE", "DURATION", "IS_INTERNSHIP", "COMPANY_IS_STAFFING"]

# Index and One-Hot Encode
indexers = [StringIndexer(inputCol=col, outputCol=f"{col}_Idx", handleInvalid="skip") for col in categorical_cols]
encoders = [OneHotEncoder(inputCol=f"{col}_Idx", outputCol=f"{col}_vec") for col in categorical_cols]


In [7]:
# combines all categorical and numeric columns into one vector coulmn
assembler = VectorAssembler(
    inputCols=continuous_cols + [f"{col}_vec" for col in categorical_cols],
    outputCol="features"
)

#create pipeline for sequential transformation
pipeline = Pipeline(stages=indexers + encoders + [assembler])
data = pipeline.fit(df_cleaned).transform(df_cleaned)

# polynomial regression for MIN_YEARS_EXPERIENCE
data = data.withColumn("MIN_YEARS_EXPERIENCE_SQ", pow(col("MIN_YEARS_EXPERIENCE"), 2))

#assemble og features plus poly feature into new feature vector
assembler_poly = VectorAssembler(
    inputCols=["features", "MIN_YEARS_EXPERIENCE_SQ"],
    outputCol="features_poly"
)
data = assembler_poly.transform(data)

#split data into training and testing sets for model evaluation
reg_train, reg_test = data.randomSplit([0.8, 0.2], seed=51)
#This split reserves 80% of the data for training and 20% for testing which provides enough data for the model to learn while keeping a reliable holdout set for evaluation.

#displays final structure with salary as target
data.select("SALARY", "features", "features_poly").show(5, truncate=False)

+------+--------------------------------------+-------------------------------------------+
|SALARY|features                              |features_poly                              |
+------+--------------------------------------+-------------------------------------------+
|192800|(9,[0,1,4,6],[6.0,55.0,1.0,1.0])      |(10,[0,1,4,6,9],[6.0,55.0,1.0,1.0,36.0])   |
|125900|(9,[0,1,4,6],[12.0,18.0,1.0,1.0])     |(10,[0,1,4,6,9],[12.0,18.0,1.0,1.0,144.0]) |
|118560|[5.0,20.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0]|[5.0,20.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,25.0]|
|192800|(9,[0,1,4,6],[6.0,55.0,1.0,1.0])      |(10,[0,1,4,6,9],[6.0,55.0,1.0,1.0,36.0])   |
|116500|(9,[0,1,4,6],[12.0,16.0,1.0,1.0])     |(10,[0,1,4,6,9],[12.0,16.0,1.0,1.0,144.0]) |
+------+--------------------------------------+-------------------------------------------+
only showing top 5 rows


## Linear Regression

In [9]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
import pandas as pd
from scipy.stats import t

lin_reg = LinearRegression(featuresCol="features", labelCol="SALARY")

lin_reg_fit = lin_reg.fit(reg_train) #fit model to training data

salary_pred = lin_reg_fit.transform(reg_test) #predicting on test data

#define the evaluators
r2_eval = RegressionEvaluator(labelCol="SALARY", predictionCol="prediction", metricName="r2")
rmse_eval = RegressionEvaluator(labelCol="SALARY", predictionCol="prediction", metricName="rmse")
mae_eval = RegressionEvaluator(labelCol="SALARY", predictionCol="prediction", metricName="mae")

#generate metrics
r2_score = r2_eval.evaluate(salary_pred)
rmse_val = rmse_eval.evaluate(salary_pred)
mae_val = mae_eval.evaluate(salary_pred)

print(f"R² Score: {r2_score:.4f}")
print(f"RMSE: {rmse_val:.2f}")
print(f"MAE: {mae_val:.2f}")


25/10/05 17:42:24 WARN Instrumentation: [edf115ff] regParam is zero, which might cause numerical instability and overfitting.


R² Score: 0.2840
RMSE: 35315.94
MAE: 27676.61


In [11]:
#getting coefs and stats
model_summary = lin_reg_fit.summary

coefs = lin_reg_fit.coefficients
intercept_val = lin_reg_fit.intercept
stderr = model_summary.coefficientStandardErrors
t_vals = model_summary.tValues
p_vals = model_summary.pValues

# Compute degrees of freedom for t-distribution
n_obs = model_summary.numInstances
num_features = len(coefs)
dfree = n_obs - num_features - 1

# Critical t-value for 95% CI
t_crit = t.ppf(0.975, dfree)

feature_labels = [
    "MIN_YEARS_EXPERIENCE",
    "DURATION",
    "IS_INTERNSHIP",
    "COMPANY_IS_STAFFING",
    "EMPLOYMENT_TYPE_NAME_A",
    "EMPLOYMENT_TYPE_NAME_B",
    "REMOTE_TYPE_NAME_X",
    "REMOTE_TYPE_NAME_Y",
    "REMOTE_TYPE_NAME_Z"
]

coef_table = pd.DataFrame({
    "Feature": ["Intercept"] + feature_labels,
    "Estimate": [intercept_val] + coefs.tolist(),
    "Std Error": stderr,
    "t-Stat": t_vals,
    "p-Value": p_vals
})

# Add confidence intervals
coef_table["95% CI Lower"] = coef_table["Estimate"] - t_crit * coef_table["Std Error"]
coef_table["95% CI Upper"] = coef_table["Estimate"] + t_crit * coef_table["Std Error"]

print("\nCoefficient Summary:")
print(coef_table)


print("\nModel Interpretation:")
print(f"- R² of {r2_score:.2f} suggests the model explains about {r2_score*100:.1f}% of salary variance.")
print(f"- RMSE of ${rmse_val:,.0f} means predictions deviate by roughly this amount on average.")
print(f"- Features with p-values < 0.05 are statistically significant in predicting salary.\n")



Coefficient Summary:
                  Feature      Estimate    Std Error     t-Stat       p-Value  \
0               Intercept  76735.577948   102.356000  66.277489  0.000000e+00   
1    MIN_YEARS_EXPERIENCE   6783.898664    23.632630  -1.831651  6.702934e-02   
2                DURATION    -43.286721  6866.446762  -1.025459  3.051684e-01   
3           IS_INTERNSHIP  -7041.257613  1063.265929  -0.595172  5.517402e-01   
4     COMPANY_IS_STAFFING   -632.826152  3011.540238  -0.421858  6.731367e-01   
5  EMPLOYMENT_TYPE_NAME_A  -1270.441524  3605.833255  -1.077711  2.811853e-01   
6  EMPLOYMENT_TYPE_NAME_B  -3886.046755  2462.498142   3.306373  9.480161e-04   
7      REMOTE_TYPE_NAME_X   8141.937756  2529.947825   3.635882  2.782402e-04   
8      REMOTE_TYPE_NAME_Y   9198.592476  3172.252546   7.636575  2.398082e-14   
9      REMOTE_TYPE_NAME_Z  24225.144004  3579.670705  21.436491  0.000000e+00   

   95% CI Lower  95% CI Upper  
0  76534.942765  76936.213130  
1   6737.574686   6830